In [ ]:
import tensorflow as tf
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv1D, Dense, BatchNormalization, MaxPooling1D, Dropout, Flatten
from tensorflow.keras.models import Sequential

%load_ext tensorboard

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
config = tf.compat.v1.ConfigProto(device_count={'CPU':20}, intra_op_parallelism_threads=8,
      inter_op_parallelism_threads=40, log_device_placement=True)
sess = tf.compat.v1.Session(config=config)

In [ ]:
integer_encoder = LabelEncoder()  
one_hot_encoder = OneHotEncoder(categories='auto')   
input_features = []
input_features = np.load('sequences.npy')
input_labels = np.load('labels.npy')

In [ ]:
os.system('rm -rf ./logs/')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(
    input_features, input_labels, test_size=0.25, random_state=42)

with sess:
    
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=12, input_shape=(train_features.shape[1], 4)))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax')) 
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=['binary_accuracy'])


    
    history = model.fit(train_features, train_labels, batch_size=100,
                                epochs=50, verbose=0, validation_split=0.25,
                                callbacks=[tensorboard_callback])

    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
    plt.show()

In [ ]:
%tensorboard --logdir logs/fit